### 1- Importing the needed libraries:

In [1]:
import requests
import time
from bs4 import BeautifulSoup as bs
import csv
import string
from itertools import zip_longest
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

### 2- Identifing the used lists:

In [2]:
brands=[]
Models= []
Prices=[]
Description=[]
links=[]
colors=[]
rams=[]
storages=[]
fulfillings=[]
batteries=[]
screen_sizes=[]
ratings=[]
rating_avgs=[]
elements=[]

### 3- Scraping the pages with filters in SOUQ.com:

In [3]:
# Tell Selenium to get the URL we're interested in.
driver = webdriver.Chrome()

# Scraping multiple pages
for x in range(10):
# Throw your source into BeautifulSoup and start parsing!
    result= driver.get(f'https://egypt.souq.com/eg-en/mobile-phone/apple%7Csamsung%7Cxiaomi%7Coppo%7Chuawei/smartphone/a-7-1870/l/?_=1628070328225&section=2&page={x+1}')
    timeout = 5
    try:
        element_present = EC.presence_of_element_located((By.ID, 'element_id'))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print(f"Page {x+1} has been scrolled down")
    source_data = driver.page_source
    soup = bs(source_data)
# scraping the model and the price of the phones from the main page
    Model= soup.find_all("h1" , {"class":"itemTitle"})
    L= soup.find_all("div" , {"class":"col col-info item-content"})
    Price= soup.find_all("h3" , {"class":"itemPrice"})
    fulfilling= soup.find_all("span" , {"class":"free-shipping fs-ab-black"})    
# Filling the lists with the cleaned elements
    for i in range(len(Model)):
        brands.append(Model[i].text.strip().split(' ')[0])
        Prices.append(Price[i].text.strip().replace("EGP", ""))
        Description.append(Model[i].text.strip())
        links.append(L[i].find("a").attrs['href'])
        try:        
            fulfillings.append(fulfilling[i].text.strip())
        except :
            fulfillings.append("Paid Shipping")

Page 1 has been scrolled down
Page 2 has been scrolled down
Page 3 has been scrolled down
Page 4 has been scrolled down
Page 5 has been scrolled down
Page 6 has been scrolled down
Page 7 has been scrolled down
Page 8 has been scrolled down
Page 9 has been scrolled down
Page 10 has been scrolled down


### 4- Exctracting more details from the inner page in each phone:

In [5]:
for link in links:

    result= driver.get(link)
    source_data = driver.page_source
    search = driver.find_element_by_xpath('//a[@class="expand readmore specs"]')
    search.click()
    time.sleep(1)
    
    try: 
        try:
            color= driver.find_element_by_xpath('//div[@class="level-1 item-connection"and @data-toggle="colors-drop"]/span')
        except :
            color= driver.find_element_by_xpath('//div[@class="connection-stand"]/div/div/div[@class="item-connection text-center active"]/a')

        colors.append(color.text.strip())
    except:
        colors.append("None")
    
    try:        
        ram= driver.find_element_by_xpath('//div[@class="connection-stand"]/div/div[@id="rams_en"]/div[@class="item-connection text-center active"]/a')
        rams.append(ram.text.replace("GB", "").strip())
    except :
        ram= "None"
        rams.append(ram)
    try:      
        storage= driver.find_element_by_xpath('//div[@class="connection-stand"]/div/div[@id="storage_capacity_en"]/div[@class="item-connection text-center active"]/a')
        storages.append(storage.text.replace("GB", "").strip())
    except :
        storage= "None"   
        storages.append(storage)
    try:
        try:        
            battery= driver.find_element_by_xpath('//dd[contains(text(),"mA")]')
        except :
            battery= driver.find_element_by_xpath('//dd[contains(text(),"milliampere_hour")]')
        batteries.append(battery.text.replace("mAH", "").replace("mAh", "").replace("milliampere_hour", "").strip())
    except:
        batteries.append("None")
    
    try:  
        try:
            screen_size= driver.find_element_by_xpath('//dd[contains(text(),"inch")]')
        except:        
            screen_size= driver.find_element_by_xpath('//dd[contains(text(),"Inch")]') 

        screen_sizes.append(screen_size.text.replace("inches", "").replace("Inch", "").strip())
    except:
        screen_sizes.append("None")
    try:
        rating_avg= driver.find_element_by_xpath('//div[@class="grouped-list"]/div/div/div[@class="product-rating clearfix clearPadding-v-small"]/div[@class="large-4 small-12 columns"]/div/div[@class="hide-for-small-only"]/div/strong')
        rating_avgs.append(rating_avg.text.strip())
    except:
        rating_avg= "None" 
        rating_avgs.append(rating_avg)
    try:
        rating= driver.find_element_by_xpath('//span[contains(text(),"ratings")]')
        ratings.append(rating.text.replace("ratings", "").strip())
    except:
        rating= "None"
        ratings.append(rating)
    try:
        try:
            Modell = driver.find_element_by_xpath('//dt[contains(text(),"Model Name")]/following-sibling::dd[1]')
        except:
            Modell = driver.find_element_by_xpath('//dt[contains(text(),"Model Number")]/following-sibling::dd[1]')

        Models.append(Modell.text.strip())
    except:
        Models.append("None")

driver.quit()



### 5- Create csv file and fill it with values:

In [6]:
# Using Zip_longest to make the columns in the right position in excel
file_list =[brands,Models,Prices,Description,links,colors,rams,storages,fulfillings,batteries,screen_sizes,rating_avgs,ratings]
exported = zip_longest(*file_list)

# Create csv file and fill it with values
with open("F:\mobiles_souq.csv","w", newline='',encoding='utf8') as myfile:
    wr=csv.writer(myfile)
    wr.writerow(["Brand","Model","Price","Description","URL","Color","RAM","Storage","Shipping","Battery","Screen","Rating","Rating_count"])
    wr.writerows(exported)
